This is an exercise in using Plotly as an alternative to plotting financial charts.

In [143]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.tools.set_credentials_file(username='username', api_key='key')

import pandas_datareader as web
import pandas as pd
import datetime as dt
import numpy as np

In [144]:
#Define your Start and End Dates
start = dt.datetime(2013,8,15)
end = dt.datetime(2018,10,24)

#new data frame fetched from morningstar 
df = web.DataReader("sq", 'yahoo', start, end).reset_index()
print(df.tail())
df2 = web.DataReader("msft", 'yahoo', start, end).reset_index()

trace = go.Candlestick(
    x = df.Date,
    open = df.Open,
    high = df.High,
    low = df.Low,
    close = df.Close
)

trace2 = go.Scatter(
    x = df2.Date,
    y = df2.Close,
    opacity = 0.6
)

data = [trace]
py.iplot(data, filename = 'simple candlestick')  #displays the chart without the initial chart layout


          Date       High        Low       Open      Close    Volume  \
733 2018-10-18  77.570000  73.599998  76.190002  74.959999  15368000   
734 2018-10-19  78.699997  74.010002  77.480003  74.070000  16924800   
735 2018-10-22  76.349998  72.279999  74.800003  75.720001  15948000   
736 2018-10-23  74.419998  71.550003  72.790001  74.150002  16838400   
737 2018-10-24  75.209999  69.099998  74.070000  69.339996  15587857   

     Adj Close  
733  74.959999  
734  74.070000  
735  75.720001  
736  74.150002  
737  69.339996  
High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~tedtran6/0 or inside your plot.ly account where it is named 'simple candlestick'


Plotting SQ and MSFT against eachother. One candlestick chart, and one line chart. 

In [145]:
chartLayout = go.Layout(
    title = "Price of SQ and MSFT",
    xaxis = dict(
        title = "Time",
        rangeslider = dict(visible = False)
    ),
    yaxis = dict(title = "Price ($)")
)

In [146]:
#displaying the new chart without the bottom slider
data = [trace, trace2]
figure = go.Figure(data = data, layout = chartLayout)
py.iplot(figure, filename = "1")

### 100 moving average
this is made from a rolling moving average. Which means it takes the most recent 100 and it calculates the average of those. 

https://www.investopedia.com/terms/d/deathcross.asp


In [147]:
df3 = web.DataReader("aapl", 'yahoo', start, end).reset_index()   #not completely sure why reset index is important, but it is
#pandas rolling method
df3["h100ma"] = df3["Adj Close"].rolling(window=100).mean()
df3["h50ma"] = df3["Adj Close"].rolling(window=50).mean()
print(df3.tail())

aapl_trace = go.Candlestick(
    name = "AAPL",
    x = df3.Date,
    open = df3.Open,
    high = df3.High,
    low = df3.Low,
    close = df3.Close
)

aapl_100ma = go.Scatter(
    name = "100ma of AAPL",
    x = df3.Date,
    y = df3.h100ma,
    line = dict(color = "#17BECF"),
    opacity = 0.6
)

aapl_50ma = go.Scatter(
    name = "50ma of AAPL",
    x = df3.Date,
    y = df3.h50ma,
    line = dict(color = "#b942f4"),
    opacity = 0.6
)

layout = go.Layout(
    title = "Price of AAPL with 50 & 100 moving Average",
    xaxis = dict(title = "Time"),
    yaxis = dict(title = "Price ($)")
)

data = [aapl_trace, aapl_100ma, aapl_50ma]
figure = go.Figure(data = data, layout = layout)
py.iplot(figure)

           Date        High         Low        Open       Close      Volume  \
1304 2018-10-18  219.740005  213.000000  217.860001  216.020004  32581300.0   
1305 2018-10-19  221.259995  217.429993  218.059998  219.309998  33078700.0   
1306 2018-10-22  223.360001  218.940002  219.789993  220.649994  28792100.0   
1307 2018-10-23  223.250000  214.699997  215.830002  222.729996  38616200.0   
1308 2018-10-24  224.229996  214.550003  222.600006  215.089996  36441623.0   

       Adj Close      h100ma       h50ma  
1304  216.020004  205.453063  220.129801  
1305  219.309998  205.777716  220.353001  
1306  220.649994  206.122047  220.615401  
1307  222.729996  206.453595  220.892601  
1308  215.089996  206.692899  220.999401  


Only plots where the shorter MA is bigger than the longer MA

In [148]:
#df3["h100ma"] = df3["Adj Close"].rolling(window=100).mean()
#df3["h50ma"] = df3["Adj Close"].rolling(window=50).mean()

df3["shorterBigger"] = np.where(df3["h100ma"] <df3["h50ma"], df3["h100ma"], np.nan)

df3NewClose = pd.DataFrame()   #creates an empty dataframe, and then populates it
df3NewClose["Date"] = df3["Date"]
df3NewClose["Open"] =  np.where(df3["h100ma"] <df3["h50ma"], df3["Open"], np.nan)
df3NewClose["High"] =  np.where(df3["h100ma"] <df3["h50ma"], df3["High"], np.nan)
df3NewClose["Low"] =  np.where(df3["h100ma"] <df3["h50ma"], df3["Low"], np.nan)
df3NewClose["Close"] =  np.where(df3["h100ma"] <df3["h50ma"], df3["Close"], np.nan)



aapl_new_trace = go.Candlestick(
    name = "AAPL",
    x = df3NewClose.Date,
    open = df3NewClose.Open,
    high = df3NewClose.High,
    low = df3NewClose.Low,
    close = df3NewClose.Close
)

aapl_shorter = go.Scatter(
    name = "shorter ma bigger",
    x = df3.Date,
    y = df3.shorterBigger,
    line = dict(color = "#b942f4"),
    opacity = 0.6
)

layout = go.Layout(
    title = "Price of AAPL only when the 50ma > 100ma",
    xaxis = dict(
        title = "Time",
        rangeslider = dict(visible = False)
    ),
    yaxis = dict(title = "Price ($)")
)

data = [aapl_new_trace, aapl_shorter]
figure = go.Figure(data = data, layout = layout)
py.iplot(figure)

In [152]:
df4 = web.DataReader("aapl", 'yahoo', start, end).reset_index()
df4["h100ma"] = df4["Adj Close"].rolling(window=100).mean()
df4["h50ma"] = df4["Adj Close"].rolling(window=50).mean()

df4["ones"] = np.nan

#df.iloc[index]
i = 0
truth = True
while i < len(df4.index):
    if(df4["h100ma"].iloc[i] < df4["h50ma"].iloc[i] and truth):
        df4["ones"].iloc[i] = df4["Close"].iloc[i]
        truth = False
    
    if(df4["h100ma"].iloc[i] > df4["h50ma"].iloc[i] and truth == False):
        df4["ones"].iloc[i] = df4["Close"].iloc[i]
        truth = True
        
    i = i+1
end

intersectTrace = go.Scatter(
    name = "crossover points",
    x = df4.Date,
    y = df4.ones,
    mode = 'markers',
    marker = dict(
        size = 10
    ),
    opacity = 0.7
)

#for i in df4["ones"]:
#    if df4["h100ma"].

layout = go.Layout(
    title = "Price of AAPL, with indicators where 50ma crosses 100ma",
    xaxis = dict(
        title = "Time",
        rangeslider = dict(visible = False)
    ),
    yaxis = dict(title = "Price ($)")
)

data = [aapl_trace, intersectTrace]
figure = go.Figure(data = data, layout = layout)
py.iplot(figure, filename = 'intersect')

C:\Users\Ted\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [171]:
portfolio = pd.DataFrame()
portfolio["Date"] = df4["Date"]
portfolio["Value"] = 0
portfolio["Shares"] = 0

i = 0
cash = 1000
shares = 0
truth = True
timesBought = 0
timesSold = 0

while i < len(portfolio.index):    
    if(df4["h100ma"].iloc[i] < df4["h50ma"].iloc[i] and truth):
        shares = cash/df4["Close"].iloc[i]
        cash = 0
        timesBought += 1
        truth = False
    
    if(df4["h100ma"].iloc[i] > df4["h50ma"].iloc[i] and truth == False):
        cash = shares * df4["Close"].iloc[i]
        shares = 0
        timesSold += 1
        truth = True
        
    portfolio["Value"].iloc[i] = cash + shares * df4["Close"].iloc[i]
        
    i = i+1
end

if truth == False:
    cash = shares * df4["Close"].iloc[i-1]
    shares = 0

print(cash)
print(timesBought)

C:\Users\Ted\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



1890.1824782258807
6


In [216]:
returns = go.Scatter(
    name = "MA Crossover Strategy",
    x = portfolio.Date,
    y = portfolio.Value,
    line = dict(color = "#ff425b"),
    stackgroup = "one"
)

buyAndHold = go.Scatter(
    name = "Buy and Hold Strategy",
    x = df4.Date,
    y = 14.10437 * df4.Close,
    line = dict(color = "#7eb8fc"),
    stackgroup = "two",
    opacity = 0.6
)
#print(portfolio.Value)

layout = go.Layout(
    title = "Returns on Different Strategies",
    xaxis = dict(title = "Time"),
    yaxis = dict(title = "Portfolio Value ($)")
)

data = [returns, buyAndHold]
figure = go.Figure(data = data, layout = layout)
py.iplot(figure, filename = "returns")